In [2]:
import pandas as pd
import ta
import ta.momentum
import ta.trend
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ta.volatility
import ta.volume
# Đọc dữ liệu từ file CSV
data = pd.read_csv('./data/OANDA_AUDCAD_Historical.csv')

data['body'] = data['close'] - data['open']
data['shadow'] = data['high'] - data['low']

ulcer = ta.volatility.UlcerIndex(close=data['close'], window=6, fillna=True)

data['ulcer'] = ulcer.ulcer_index()




# columns = ["time","Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_DIFF","isIncrease"]
# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = './train-data/OANDA_AUDCAD_Historical_Updated.csv'
data.to_csv(output_path, index=False)

In [3]:
import pandas as pd

file_path = 'data/OANDA_XAUUSD_Historical.csv'
# Đọc tệp CSV
df = pd.read_csv(file_path)

# Xóa các hàng từ 0 đến 100000
df = df.drop(df.index[0:111111])

# Lưu lại DataFrame đã xử lý vào tệp CSV mới
df.to_csv(file_path, index=False)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
import pickle

# Giả sử bạn đã có dữ liệu dưới dạng DataFrame 'df'
df = pd.read_csv('./train-data/OANDA_XAUUSD_Historical_Updated.csv')

# Chia dữ liệu thành các đặc trưng (features) và nhãn (target)
X = df[['RSI', 'STOCH_K', 'STOCH_D', 'ROC_RATE', 'ATR']]
y = df['Target']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tạo mô hình hồi quy logistic
model = LogisticRegression()

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)
# joblib.dump(model, 'logistic_model.joblib')
# with open('./scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [17]:
import pandas as pd

# Đọc file Excel
file_path = './train-data/BINANCE_BTCUSDT_Historical_Updated.csv'
df = pd.read_csv(file_path)

# Chuyển đổi cột ngày tháng (giả sử cột ngày tháng tên là 'NgayThang')
df['time'] = pd.to_datetime(df['time'])
# Thêm cột 'Thu' dựa trên cột ngày tháng
df['Thu'] = df['time'].dt.day_name()

# Lọc dữ liệu theo thứ (ví dụ: lọc ra các ngày là Thứ Hai)
df_sunday = df[df['Thu'] == 'Sunday']

# Lưu dữ liệu lọc ra thành một file Excel mới (nếu cần)
df_sunday.to_csv('BINANCE_BTCUSDT_Historical_Updated_Sunday.csv', index=False)


In [5]:
import pandas as pd
from datetime import datetime
import pytz

# Đọc dữ liệu từ file CSV
data = pd.read_csv('./train-data/OANDA_XAUUSD_Historical_Updated.csv')

# Hàm chuyển đổi một chuỗi thời gian về giờ UTC
def convert_to_utc(time_str):
    local_time = datetime.fromisoformat(time_str)
    return local_time.astimezone(pytz.utc)

# Áp dụng hàm convert_to_utc cho từng giá trị trong cột 'time'
data['time'] = data['time'].apply(convert_to_utc)

# Nếu bạn muốn lưu dưới dạng chuỗi thời gian ISO 8601 sau khi chuyển đổi
data['time'] = data['time'].apply(lambda x: x.isoformat())

# columns = ["time","Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_DIFF","isIncrease"]
# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = './train-data/OANDA_XAUUSD_Historical_Updated_1.csv'
data.to_csv(output_path, index=False)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib
symbol = 'XAUUSD'
# df = pd.read_csv('./train-data/OANDA_XAUUSD_Historical.csv')
df = pd.read_csv(f'./train-data/OANDA_{symbol}_Historical.csv')

# Xử lý giá trị thiếu
df.dropna(inplace=True)
features = []
# Số lượng giá trị tiếp theo cần tính trung bình
bodies = [f'body_{i}' for i in range(0, 50)]    

# Kết hợp các cột hiện có và các cột tự động tạo
features += bodies


# Chọn đặc trưng và biến mục tiêu
X = df[features]
y = df['target_body']
# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Khởi tạo và huấn luyện mô hình
rf_classifier = RandomForestRegressor(random_state=42)
rf_classifier.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = rf_classifier.predict(X_test)

# Dự đoán giá tiếp theo
latest_data = X.iloc[-1].values.reshape(1, -1)
next_movement = rf_classifier.predict(latest_data)
model_filename = './random_forest_xauusd_model_target_body.pkl'


joblib.dump(rf_classifier, model_filename)
if next_movement[0] > 0:
    print("Giá dự kiến sẽ TĂNG vào ngày tiếp theo.")
else:
    print("Giá dự kiến sẽ GIẢM vào ngày tiếp theo.")

In [19]:
import pandas as pd
import numpy as np
time = 240
round_num = 5
# Đọc dữ liệu từ file CSV
new_data = pd.read_csv(f'./data/OANDA_XAUUSD_{time}m_Historical.csv')

# new_data['target'] = np.where(new_data['close'] > new_data['open'], 1, 0)
# Tạo các đặc trưng từ nến trước (ví dụ với nến hiện tại và nến trước đó)
new_data['Price_Change'] = round(new_data['close'].pct_change(), 5)  # % thay đổi giá
new_data['Candle_Length'] = round(new_data['close'] - new_data['open'] , 5) # Chiều dài của nến
new_data['Volatility'] = round(new_data['high'] - new_data['low'], 5)  # Biến động giá
new_data['Future_Change'] = np.where(new_data['close'].shift(-1) > new_data['close'], 1, 0)
new_data.dropna(inplace=True)

output_path = f'./train-data/OANDA_XAUUSD_{time}m_Historical.csv'
new_data.dropna().to_csv(output_path, index=False)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib
symbol = 'XAUUSD'
indicator = 'NORMAL'
# df = pd.read_csv('./train-data/OANDA_XAUUSD_Historical.csv')
df = pd.read_csv(f'./train-data/OANDA_{symbol}_15m_Historical.csv')

# Xử lý giá trị thiếu
df.dropna(inplace=True)

features = ['Price_Change', 'Candle_Length', 'Volatility']

# Chọn đặc trưng và biến mục tiêu
X = df[features]
y = df['Future_Change']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Khởi tạo và huấn luyện mô hình
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = rf_classifier.predict(X_test)

# Dự đoán giá tiếp theo
latest_data = X.iloc[-1].values.reshape(1, -1)
next_movement = rf_classifier.predict(latest_data)
model_filename = f'./random_forest_xauusd_model_{indicator}.pkl'


joblib.dump(rf_classifier, model_filename)
if next_movement[0] > 0:
    print("Giá dự kiến sẽ TĂNG vào ngày tiếp theo.")
else:
    print("Giá dự kiến sẽ GIẢM vào ngày tiếp theo.")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Giá dự kiến sẽ GIẢM vào ngày tiếp theo.
